In [1]:
from file_interaction import get_blobs

In [2]:
from datetime import datetime, timedelta
from azure.storage.blob import BlobClient, ContainerClient
from pathlib import Path
import logging
from helper import load_dotenv
from tqdm.auto import tqdm
import random

In [3]:
dotenv = load_dotenv()

In [4]:
def get_completed_directories( files, available_blobs ):
    directory_entries = {}

    for f in files:
        if f[1] not in directory_entries:
            directory_entries[f[1]] = {
                'files' : 0,
                'azure' : 0
            }
    
        directory_entries[f[1]]['files'] += 1
    
    for ab in available_blobs:
        prefix = '/'.join(ab.split('/')[:-1]) + '/'
    
        if prefix not in directory_entries:
            directory_entries[prefix] = {
                'files' : 0,
                'azure' : 0
            }
    
        directory_entries[prefix]['azure'] += 1

    all_entries = list(directory_entries.keys())
    repeat_processing = True
    
    while repeat_processing:
        length_before = len(all_entries)
        all_entries = list(set(
            all_entries + ["/".join(entry.split('/')[:-2]) + "/" for entry in all_entries]
        ))
        length_after = len(all_entries)
        
        if length_before == length_after:
            repeat_processing = False
    
    completed_directories = { key : False for key in all_entries }
    
    for key in directory_entries:
        completed_directories[key] = directory_entries[key]['files'] == directory_entries[key]['azure']
    
    dict_changed = True
    
    while dict_changed:
        dict_changed = False
        
        for key in completed_directories:
            if completed_directories[key] == False:
                relevant_keys = [dir_path for dir_path in completed_directories if key in dir_path and key != dir_path]
                
                if len(relevant_keys) > 0 and False not in [completed_directories[rk] for rk in relevant_keys]:
                    completed_directories[key] = True
                    dict_changed = True
    
    return sorted([key for key in completed_directories if completed_directories[key] == True])

In [5]:
def get_blobs( filter=None ):
    container = ContainerClient.from_connection_string(conn_str=connection_string, container_name=container_name)
    
    if filter:
        return list(container.list_blob_names( name_starts_with=filter ))

    return list(container.list_blob_names())

In [6]:
def download_blob( filepath, blob_name ):
    pass

In [7]:
def upload_blob( filepath, prefix_path ):
    blob = BlobClient.from_connection_string(conn_str=connection_string, container_name=container_name, blob_name=f"{ prefix_path }{ filepath.name }")

    with filepath.open("rb") as data:
         blob.upload_blob(
             data,
             connection_timeout=600
         )

In [8]:
connection_string = f'DefaultEndpointsProtocol=https;AccountName={ dotenv["AZURE_ACCOUNT_NAME"] };AccountKey={ dotenv["AZURE_ACCOUNT_KEY"] };EndpointSuffix=core.windows.net'
container_name = dotenv['AZURE_CONTAINER_NAME']

In [9]:
BASE_DIRS = [
    Path('/Users/frederic.birwe/Desktop/azure_transfer/'),
    #Path('/Volumes/MAT-Share/Technologie_Entwicklung/aktuell/__frederic/masterarbeit/')
]

In [10]:
def transform_filepath( f ):
    out = str(f)

    for bdir in BASE_DIRS:
        out = out.replace(str(bdir) + '/', '')

    return out.replace(f.name,'')

In [11]:
files = []
for bdir in BASE_DIRS:
    print(bdir.name)

    for dir_entry in bdir.iterdir():
        if dir_entry.is_dir() and dir_entry.name != 'temp':
            for f in tqdm(dir_entry.glob('./**/*')) :
                if f.is_dir() == False and f.name.startswith('.') == False:
                    files.append(f)

files = [
    (f, transform_filepath(f), f.name)
    for f in files
]

azure_transfer


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [12]:
available_blobs = get_blobs()
files_to_upload = [f for f in files if f[1]+f[2] not in available_blobs]
#random.shuffle(files_to_upload)

In [13]:
get_completed_directories( files, available_blobs )

['data/141607/halftone600dpi/',
 'data/145175/',
 'data/145175/halftone600dpi/',
 'data/145175/pdf/',
 'data/145175/ps2400dpi150lpi/',
 'data/145175/vps2400dpi150lpi/',
 'data/23-10-03_Testformen/',
 'data/23-10-03_Testformen/halftone600dpi/',
 'data/23-10-03_Testformen/pdf/',
 'data/23-10-03_Testformen/ps2400dpi150lpi/',
 'data/23-10-03_Testformen/vps2400dpi150lpi/',
 'data/23-10-18_farbe/',
 'data/23-10-18_farbe/halftone600dpi/',
 'data/23-10-18_farbe/pdf/',
 'data/23-10-18_farbe/ps2400dpi150lpi/',
 'data/23-10-18_farbe/vps2400dpi150lpi/',
 'data/23-10-19_farbe/',
 'data/23-10-19_farbe/halftone600dpi/',
 'data/23-10-19_farbe/pdf/',
 'data/23-10-19_farbe/ps2400dpi150lpi/',
 'data/23-10-19_farbe/vps2400dpi150lpi/',
 'data/24-01-02_randomPages/',
 'data/24-01-02_randomPages/halftone600dpi/',
 'data/24-01-02_randomPages/pdf/',
 'data/24-01-02_randomPages/ps2400dpi150lpi/',
 'data/24-01-02_randomPages/vps2400dpi175lpi/',
 'data/508001/halftone600dpi/',
 'data/508001/pdf/',
 'data/508001/p

In [14]:
f'{ (len(files) - len(files_to_upload)) }/{ len(files) }, { ((len(files) - len(files_to_upload)) / len(files)) * 100 }%'

'51414/51414, 100.0%'

In [15]:
0 / 0

ZeroDivisionError: division by zero

In [ ]:
for f,prefix,filename in tqdm(files_to_upload):
    upload_blob( f, prefix )

In [28]:
data_dir = dotenv['DATA_DIRS'][0]

data_dirs = [entry for entry in data_dir.iterdir() if entry.is_dir()]

In [29]:
for dd in data_dirs:
    blobs = get_blobs(f'data/{ dd.name }')
    entries = [entry for entry in dd.glob('./**/*') if entry.is_dir() == False and entry.name.startswith('.') == False]
    ready_for_deletion = False not in [str(entry).replace(str(data_dir.parent) + '/','') in blobs for entry in entries]

    print( dd.name, ready_for_deletion, len(blobs), len(entries) )

00_to_sort True 0 0
23-10-19_farbe True 3152 3152
24-01-02_randomPages True 1147 1147
23-10-18_farbe True 12896 12896
23-10-03_Testformen True 315 315
